In [1]:
# 生のデータファイルの置き場所
# dockerの場合
path_to_RawData_folder = '/home/biofunc/data/ラット行動実験データ（生体機能・組織培養室PC）/Data'

dict_of_target = {
   'DIFF':{
        'subjects':['SH010', 'SH012', 'SH014', 'SH015', 'SH017', 'SH022', 'SH024'],
        'MSNs':['3b_disc-Diff', 'AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Diff_rft'],
        'Index':['X', 'Y']
    },
    'MIXED':{
        'subjects':['I321', 'I322', 'I325', 'SH011', 'SH013', 'SH020'],
        'MSNs':['3a_disc-Mixed', 'AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Mixed_rft'],
        'Index':['X', 'Y']
    }
}

window_size = [10, 20, 150]

# ↑　プログラムを動かすたびに編集するところ　↑

In [2]:
%pip install japanize-matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
import glob
import os
from pathlib import Path
import shutil
import re
from datetime import datetime as dt
import numpy as np
import pandas as pd
#とりあえず、json型で出力することにした
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#例えば、seaborn を利用している場合であれば sns.set() などで描画フォントが seaborn のデフォルトに上書きされ、日本語表示がされなくなります。
#sns.set(font="IPAexGothic") のように利用フォントに IPAexGothic を設定するか、フォント上書き後に japanize_matplotlib.japanize() を利用するなどで日本語表示できるはずです。
import japanize_matplotlib
japanize_matplotlib.japanize()

#############################################################################
# 参考：jsonの構造
# ラット一匹につき、jsonファイルは一つ
{ 
    'rat_name':{

        'session_1':{
            "Start Date": "10/04/22",
            "End Date": "10/04/22",
            "Subject": "sh010",
            "MSN": "1b_mag-pel",
            "A": "3.000",
            "E": [
                ["0", "3.000", "3.000", "2.000", "2.000", "4.000"],
                ["5", "1.000", "1.000", "2.000", "1.000", "0.000"],
            ]
        },

        'session_2':{
            "Start Date": "10/05/22",
            "End Date": "10/05/22",
            "Subject": "sh010",
            "MSN": "1b_mag-pel",
            "A": "3.000",
            "E": [
                ["0", "3.000", "3.000", "2.000", "2.000", "4.000"],
                ["5", "1.000", "1.000", "2.000", "1.000", "0.000"],
            ]
        }
    }
}
#############################################################################

def make_output_folder(path_to_RawData_folder):
    # 出力フォルダの作成
    # 出力フォルダのパスを作る
    output_timestamp = str(dt.now().strftime('%Y%m%d_%Hh%Mm%Ss'))
    path_to_output_folder = f'{Path(path_to_RawData_folder).parent}/Output/{output_timestamp}_ラット行動実験データ'
    if not os.path.exists(path_to_output_folder):
        os.makedirs(path_to_output_folder)
    return path_to_output_folder


# 生のデータファイルを読み込み、json型に変換する関数
def make_json_from_RawData(path_to_a_RawData_folder, path_to_output_folder):
    #ファイルを読み込み
    path_of_all_raw_data = sorted(glob.glob(path_to_a_RawData_folder + '/*'))
    # さらに、サブディレクトリを除去
    path_of_all_raw_data = [a_path for a_path in path_of_all_raw_data if not os.path.isdir(a_path)]

    #データ項目の正規表現パターン
    pattern_alphabet_index = '(^[a-zA-Z]+[a-zA-Z|\s]*)\W.*'  #先頭がアルファベットの場合、最初の「:」までの文字列
    pattern_number_index = '^[\s]+([0-9]+)\W.*'        #先頭が数字の場合、最初の「:」までの文字列
    pattern_body = '^[\w|\s]+\W+(.*)[\r\n|\n|\r]'   #最初の「:」から、改行までの文字列

    #jsonの出力ファイルの保存先を作る
    path_to_json_data_folder = f'{path_to_output_folder}/json_from_RawData'
    if os.path.exists(path_to_json_data_folder):
        shutil.rmtree(path_to_json_data_folder)
        os.mkdir(path_to_json_data_folder)
    elif  not os.path.exists(path_to_json_data_folder):
        os.mkdir(path_to_json_data_folder)
    
    # ラット一匹分のデータを取り込み、json型に変換する
    for path_of_a_raw_data in path_of_all_raw_data:
        # 一匹分のデータを格納するリスト
        output_of_a_raw_data = []
        # ファイル名からラット名を取得
        name_of_the_rat = os.path.basename(path_of_a_raw_data).split('.')[0].upper()
        # ファイルを開く
        
        with open(path_of_a_raw_data, 'r') as a_raw_data:
            # ファイルを一行ずつ読み込み
            list_a_raw_data = a_raw_data.readlines()
            # ファイルの先頭にある「Start Date:」の行番号を取得
            index_for_crop = [i for i, a_row in enumerate(list_a_raw_data) if a_row.startswith('Start Date:')]
            index_for_crop = index_for_crop + [len(list_a_raw_data) + 1]
            # ファイルの先頭にある「Start Date:」の行番号を基準に、
            # ファイルをセッションごとに分割
            list_a_raw_data = [list_a_raw_data[index_for_crop[i]:index_for_crop[i+1]] for i in range(len(index_for_crop)-1)]

            # 一セッション分のデータを取り込み、json型に変換する
            for a_session_data in list_a_raw_data:
                # 一セッション分のデータを格納するリスト
                list_a_session_data = []
                # 一セッション分のデータを一行ずつ読み込み
                for a_row in a_session_data:
                    # 一行分のデータを格納するリスト
                    list_a_row_data = []
                    # 一行分のデータを正規表現で分割
                    alphabet_index = re.findall(pattern_alphabet_index, a_row)
                    number_index = re.findall(pattern_number_index, a_row)
                    a_body = re.findall(pattern_body, a_row)

                    # 以下の場合分けの概要
                    # 1) 行名がアルファベットの場合
                        # a) 値の場合       →　そのまま代入
                        # b) 値なしの場合   →　次の行から行列がはじまる
                    # 2) 行名が数字の場合
                        # a) 行列     →　行列を統合する

                    # 行名がアルファベットの場合
                    if alphabet_index != []:
                        list_a_row_data += alphabet_index
                        # 値が存在するときは、
                        # 値は行列ではありえないので、素直に挿入
                        if a_body !=['']:
                            list_a_row_data += a_body
                        # 値が存在しないときは、
                        # 次の行から行列がはじまるので、空のリストを挿入
                        elif a_body ==['']:
                            list_a_row_data += [[]]
                        list_a_session_data += [list_a_row_data]
                    # 行名が数字の場合
                    elif number_index != []:
                        # 行列の場合は、行列を統合する
                        a_new_row = number_index + a_body[0].split()
                        # 「list_a_session_data」の一番最後のリストに、どんどんブッ込んで行く
                        # そして、次の行名(アルファベット)が間にはさまるので、別の行列同士がくっつく事はない
                        list_a_session_data[-1][1] += [a_new_row]
                    # 場合分け終了 

                # 一セッション分のデータ処理が終わったので、個体のデータに統合
                output_of_a_raw_data += [list_a_session_data]
            # 一匹分のデータ処理が終わり
            # [[key, value], [key, value], [], ....]　から　{key:value, key:value, :, ......}　に変換
            output_of_a_raw_data = [dict(i) for i in output_of_a_raw_data]

            # セッションの開始日時を取得する関数
            def get_date(a: dict):
                #同じ日付のファイルがる場合、上書きされてしまうので、時間もふくめてファイル名をつける
                start_time = dt.strptime(a["Start Date"] + a["Start Time"], '%m/%d/%y%H:%M:%S') #.strftime('%Y-%m-%d_%H:%M:%S')
                return(start_time)
            
            # [{session_1}, {session_2}, {}, {}, ....] から　{subject: {title_1:session_1, title_2:session_2, :, :, ....}}　に変換
            output_of_a_raw_data = [[f'{get_date(a_session)}_{a_session["MSN"]}', a_session] for a_session in output_of_a_raw_data]
            output_of_a_raw_data = {name_of_the_rat: dict(output_of_a_raw_data)}
            # json型に変換
            with open(f'{path_to_json_data_folder}/{name_of_the_rat}.json' ,'w') as output_file:
                json.dump(output_of_a_raw_data, output_file, indent=4, ensure_ascii=False)  # これを、そのままjson型として保存する
            print(f'{name_of_the_rat}: \tis converted to json format.')

# (新)MED-PC操作ミス等のエラーを修復する関数
def repaire_known_issue_of_json(path_to_output_folder):
    path_to_json_folder = f'{path_to_output_folder}/json_from_RawData'

    ##############################################################################################
    # I321、R322、R323、R324、R325のMSNによる列数の違いを修復する。

    #jsonファイルの中身を引越しする前に、この処理をするということにする。
    # 修復の処理を順不同にするには、、？
    # いや、引越しの場合データの置き場所が変わるので順不同にはできない。

    before_edited_json_subject_session_dict = {
        'I321': 'AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Mixed_rft',
        'R322': 'AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Mixed_rft',
        'R323': 'AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Diff_rft',
        'R324': 'AuX_Disc2(pretrain_sgl_lev_Mixed_rft)[self-started]',
        'R325': 'AuX_Disc2(pretrain_sgl_lev_Mixed_rft)[self-started]'
    }

    for a_subject_key in     before_edited_json_subject_session_dict.keys():
        for a_session_key in     before_edited_json_subject_session_dict[a_subject_key]:
            # 修復するjsonファイルが存在するかどうか。
            if not os.path.exists(f'{path_to_json_folder}/{a_subject_key}.json'):
                pass
            # 修復するjsonファイルが存在する場合
            elif os.path.exists(f'{path_to_json_folder}/{a_subject_key}.json'):
                with open(f'{path_to_json_folder}/{a_subject_key}.json', 'r') as f:
                    a_json = json.load(f)
                    new_json = {}

                    # 一匹分のデータ処理(普通は一匹だけしか保存されてないはず)
                    for a_subject_key in a_json.keys():
                        # 一セッション分のデータ処理
                        for a_session_key in a_json[a_subject_key].keys():
                            if a_json[a_subject_key][a_session_key]['MSN'] == before_edited_json_subject_session_dict[a_subject_key]:
                                df_X = pd.DataFrame(a_json[a_subject_key][a_session_key]['X']).astype('float64')
                                df_Y = pd.DataFrame(a_json[a_subject_key][a_session_key]['Y']).astype('float64')

                                # 0で埋まっているデーブルの場合、刺激=0、レバーチョイス＝0ということになってしまい、
                                # 「正解」と判定されてしまう。
                                index_of_last_row = 0
                                for (index_df_X, row_df_X), (index_df_Y, row_df_Y) in zip(df_X.iterrows(), df_Y.iterrows()):
                                    if row_df_X[1:].sum() + row_df_Y[1:].sum() != 0:
                                        index_of_last_row = index_df_X
                                    elif row_df_X[1:].sum() + row_df_Y[1:].sum() == 0:
                                        pass
                                df_X = df_X.iloc[:index_of_last_row+1, :]
                                df_Y = df_Y.iloc[:index_of_last_row+1, :]
                                #df_Xとdf_Yの列数は、「行番号の列」と「データの列」の合計
                                # データの列が５列の場合は、合計６列なので、
                                # 欠損しているデータ列として、「報酬の種類」の列を追加する
                                if len(df_X.columns) == 6:
                                    # X6の列の場合は、emptyなので0を入れるだけ。ソートも必要ない
                                    df_X.insert(6, 6, 0.0)
                                    if len(df_Y.columns) == 6:
                                        # 3はないことに注意！
                                        df_Y.columns = [0, 1, 2, 4, 5, 6]
                                        #正解はとりあえずペレット(=0)一択として代入した
                                        df_Y.loc[df_Y[2] <= 2, 3] = (df_Y[1]==df_Y[2])*0 | (df_Y[1]!=df_Y[2])+1
                                        ####################
                                        #print('check point 1')
                                        #display(df_Y)
                                        ####################
                                        # ソート
                                        df_Y.sort_index(axis=1, inplace=True)
                                        # jsonに戻す。valueだけだとndarrayになって以下のエラーが出るので、tolist()でlistに変換する
                                        # TypeError: Object of type ndarray is not JSON serializable
                                        a_json[a_subject_key][a_session_key]['X'] = df_X.values.tolist()
                                        a_json[a_subject_key][a_session_key]['Y'] = df_Y.values.tolist()
                                        # 修復したものを新しいjsonに保存
                                        new_json.update({a_session_key:a_json[a_subject_key][a_session_key]})
                                # XとYがそれぞれ７列のときは、何もしない
                                elif len(df_X.columns) == 7:
                                    if len(df_Y.columns) == 7:
                                        new_json.update({a_session_key:a_json[a_subject_key][a_session_key]}) 
                                #XとYがそれぞれ違う列数のときは、エラーを出す
                                else:
                                    print(f'the number of X and Y columns are different. X:{len(df_X.columns)}, Y:{len(df_Y.columns)}')
                                    print(f'{a_subject_key}, {a_session_key}')
                                    print(f'{a_json[a_subject_key][a_session_key]["MSN"]}')
                            else:
                                new_json.update({a_session_key:a_json[a_subject_key][a_session_key]})
                    # 一匹分のデータ処理が終わったら、新しいjsonファイルを保存
                    # ほんとはファイル名にrepairedをつけたいが、次の処理で処理でpath名を指定するために、つけないでおく
                    # 正規表現で指定できるなら、つけてもいいかも
                    with open(f'{path_to_json_folder}/{a_subject_key}.(6th_Columns_Added).json', 'w') as f:
                        json.dump({a_subject_key:new_json}, f, indent=4, ensure_ascii=False)
                        os.remove(f'{path_to_json_folder}/{a_subject_key}.json')
                        print(f'{a_subject_key}: \thas been added 6th column to X and Y')
    # I321終わり
    ##############################################################################################

    ##############################################################################################
    #保存先を間違ったセッションデータの引越し・新規保存、間違いjsonの削除
    Tsubject_Fsubject_THEsession_dict = {
        'I322':{'R322':'2022-09-07 11:50:12_AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Mixed_rft'},
        'I323':{'R323':'2022-09-07 12:54:45_AuX_Disc4(train)_Hi-Lef_Lo-Rig_w_correction_Diff_rft'},
        'I324':{'R324':'2022-09-07 14:04:32_AuX_Disc2(pretrain_sgl_lev_Mixed_rft)[self-started]'},
        'I325':{'R325':'2022-09-07 14:50:42_AuX_Disc2(pretrain_sgl_lev_Mixed_rft)[self-started]'},
        'SH011':{'SH010':'2022-10-22 15:43:27_3a_disc-Mixed'},
        'SH017':{'SH016':'2022-11-29 16:16:20_3b_disc-Diff'}
    }

    for a_Tsubject in Tsubject_Fsubject_THEsession_dict.keys():
        # 単純にdict.key()で値を取り出すと、dict_keys型になってしまうので、list()でlist型に変換する
        a_Fsubject = list(Tsubject_Fsubject_THEsession_dict[a_Tsubject].keys())[0]
        missed_session = Tsubject_Fsubject_THEsession_dict[a_Tsubject][a_Fsubject]

        # 目的のjsonファイルのパス

        #path_to_True_json = f'{path_to_json_folder}/{a_Tsubject}.json'
        #path_to_False_json = f'{path_to_json_folder}/{a_Fsubject}.json'
        path_to_True_json = glob.glob(f'{path_to_json_folder}/{a_Tsubject}' + '.*')
        path_to_False_json = glob.glob(f'{path_to_json_folder}/{a_Fsubject}' + '.*')
        
        # 目的のjsonファイルが両方存在しない場合は、エラーを出す
        if (len(path_to_True_json) > 1) or (len(path_to_False_json) > 1):
            print(f'{a_Tsubject}と{a_Fsubject}のjsonファイルが複数あります。')
            break
        elif (len(path_to_True_json) == 0) or (len(path_to_False_json) == 0):
            print(f'{a_Tsubject}と{a_Fsubject}のjsonファイルがありません。')
            break
        else:
            path_to_True_json = path_to_True_json[0]
            path_to_False_json = path_to_False_json[0]

        # 目的のjsonファイルが両方存在する場合は、jsonを読み込む
        if os.path.exists(path_to_True_json) and os.path.exists(path_to_False_json):
            with open(path_to_True_json, 'r') as f:
                json_True = json.load(f)
            with open(path_to_False_json, 'r') as f:
                json_False = json.load(f)

            for a_subject_of_json_False in json_False.keys():
                if a_subject_of_json_False.upper() == a_Fsubject.upper():
                    json_of_missed_session = json_False[a_Fsubject][missed_session]
                    for a_subject_of_json_True in json_True.keys():
                        if a_subject_of_json_True.upper() == a_Tsubject.upper():

                            json_True[a_Tsubject].update({missed_session:json_of_missed_session})
                            list_of_sorted_session_keys_of_Tsubject = sorted(list(json_True[a_Tsubject].keys()))
                            json_True_repaired = {a_Tsubject:{i:json_True[a_Tsubject][i] for i in list_of_sorted_session_keys_of_Tsubject}}

                            json_False_cleaned = {a_Fsubject:{i:json_False[a_Fsubject][i] for i in json_False[a_Fsubject].keys() if i != missed_session}}

                    with open(f'{path_to_json_folder}/{a_Tsubject}.(repaired).json', 'w') as f:
                        json.dump(json_True_repaired, f, indent=4, ensure_ascii=False)
                        os.remove(path_to_True_json)
                    with open(f'{path_to_json_folder}/{a_Fsubject}.(cleaned).json', 'w') as f:
                        json.dump(json_False_cleaned, f, indent=4, ensure_ascii=False)
                        os.remove(path_to_False_json)
                    print(f'{a_Tsubject} \t& {a_Fsubject} \tare repaired !!')

        
    #保存先ミスのデータ引越し終わり
    ##############################################################################################

# (新)jsonファイルをxlsxファイルに変換する関数
def make_xlsx_about_DOE_from_json(path_to_output_folder, dict_of_target, window_size):
    # jsonファイルのパスを取得
    path_to_json_folder = f'{path_to_output_folder}/json_from_RawData'
    list_of_path_to_all_json = glob.glob(f'{path_to_json_folder}/*.json')

    # 保存先のフォルダを作成(with Null)
    path_to_xlsx_folder = f'{path_to_output_folder}/xlsx_from_json.(window{window_size}_withNULL)'
    if os.path.exists(path_to_xlsx_folder):
        shutil.rmtree(path_to_xlsx_folder)
        os.mkdir(path_to_xlsx_folder)
    elif not os.path.exists(path_to_xlsx_folder):
        os.mkdir(path_to_xlsx_folder)
    
    # 保存先のフォルダを作成(without Null)
    path_to_xlsx_folder_nosepoke_isnot_false = f'{path_to_output_folder}/xlsx_from_json.(window{window_size}_withoutNULL)'
    if os.path.exists(path_to_xlsx_folder_nosepoke_isnot_false):
        shutil.rmtree(path_to_xlsx_folder_nosepoke_isnot_false)
        os.mkdir(path_to_xlsx_folder_nosepoke_isnot_false)
    elif not os.path.exists(path_to_xlsx_folder_nosepoke_isnot_false):
        os.mkdir(path_to_xlsx_folder_nosepoke_isnot_false)
    
    #このフォルダに関する処理を開始
    df_of_this_folder = pd.DataFrame()
    df_of_this_folder_nosepoke_isnot_false = pd.DataFrame()
    # このMSNの種類ごとの処理を開始
    for a_type_of_experiment in dict_of_target.keys():
        ############################################################
        display(a_type_of_experiment)
        ############################################################
        #この実験の対象となるsubjectの名前を取得
        list_of_the_subjects = [i.upper() for i in  dict_of_target[a_type_of_experiment]['subjects']]
        #この実験の対象となるMSNの名前を取得
        list_of_the_MSNs = dict_of_target[a_type_of_experiment]['MSNs'] 

        
        #json単位の処理を開始
        df_of_this_type_of_experiment = pd.DataFrame()
        df_of_this_type_of_experiment_nosepoke_isnot_false = pd.DataFrame()
        # この実験の対象となるjsonファイルのパスを取捨選択
        list_of_path_to_target_json = []
        for a_path in list_of_path_to_all_json:
            if os.path.basename(a_path).split('.')[0].upper() in list_of_the_subjects:
                list_of_path_to_target_json.append(a_path)
            else:
                pass
        # 対象となるjsonを一つずつ読み込んで、処理していく
        for a_path in list_of_path_to_target_json:
            with open(a_path, 'r') as f:
                a_json = json.load(f)

                # このjsonファイルに保存されているsubjectの名前を取得。（通常は1つだけ）
                for a_subject_key in a_json.keys():
                    ############################################################
                    #display(f'check point {a_subject_key}')
                    ############################################################
                    df_of_this_subject = pd.DataFrame()
                    df_of_this_subject_nosepoke_isnot_false = pd.DataFrame()
                    
                    first_datetime_of_this_MSN_session = 0
                    is_this_first_session_of_a_MSN = True
                    list_of_df_to_add_columns = []

                    # このsubjectのsessionを取得。（通常はいっぱい）
                    for a_session_key in a_json[a_subject_key].keys():
                        ############################################################
                        #display(f'check point {a_session_key}')
                        ############################################################
                        MSN_of_this_session = a_json[a_subject_key][a_session_key]['MSN']
                        # このsessionのMSNが、対象となるMSNであるかを判定
                        if MSN_of_this_session in list_of_the_MSNs:
                            df_of_this_session = pd.DataFrame()
                            df_of_this_session_nosepoke_isnot_false = pd.DataFrame()
                            # このsessionのjsonを取得
                            json_of_this_session = a_json[a_subject_key][a_session_key]
                            # このsessionの開始時刻と終了時刻を取得
                            start_datetime_of_this_session = dt.strptime(json_of_this_session['Start Date'] + json_of_this_session['Start Time'], '%m/%d/%y%H:%M:%S')
                            end_datetime_of_this_session = dt.strptime(json_of_this_session['End Date'] + json_of_this_session['End Time'], '%m/%d/%y%H:%M:%S')
                            # あるMSNでのsessionの内、これが一番最初のsessionかどうかを判定し、開始時刻を記録
                            if is_this_first_session_of_a_MSN == True:
                                first_datetime_of_this_MSN_session = start_datetime_of_this_session
                                is_this_first_session_of_a_MSN = False
                        
############################################################
# DOEの弁別訓練のための処理
                            df_X = pd.DataFrame(json_of_this_session['X']).astype(float)
                            df_Y = pd.DataFrame(json_of_this_session['Y']).astype(float)

                            index_of_last_row = 0
                            for (index_df_X, row_df_X), (index_df_Y, row_df_Y) in zip(df_X.iterrows(), df_Y.iterrows()):
                                if row_df_X[1:].sum() + row_df_Y[1:].sum() != 0:
                                    index_of_last_row = index_df_X
                                elif row_df_X[1:].sum() + row_df_Y[1:].sum() == 0:
                                    pass
                            df_X = df_X.iloc[:index_of_last_row+1, :]
                            df_Y = df_Y.iloc[:index_of_last_row+1, :]
                            df_X[4] = df_X[4] * 0.01

                            '''
                            Data matrix of behavior (X, Y)
                            1: Order of trial (order of nose-poke illuminations)
                            2: Nose-poking in 20 min (0 = yes; 1 = no --> session termination)
                            3: Nose-poking latency
                            4: Nose-poking duration required
                            5: Nose-poking duration (measured)
                            6: empty

                            7: Sd presented
                            8: Lever pressed (0 = left, 1 = right, 2 = no-choice, 999 = immature trial)
                            9: Outcome (0 = pellet, 1 = liquid, 2 = incorrect, 888 = no-choice, 999 = immature trial)
                            10: Sd presentation duration (Sd onset --> choice)
                            11: Reaction time (Levers presentation --> choice)
                            12: Lever choice pattern (0 = stay, 1 = shift, 999 = none)
                            '''
                            # df_Xとdf_Yの値を処理しやすい形式に計算する。列名は小文字にする。
                            df_to_add = pd.DataFrame()
                            # 1セッションあたりの時間 * 行番号 = このセッション開始からの経過時間
                            df_to_add['trial_time_stamp'] = pd.Series((df_X[1] - 1) * ((end_datetime_of_this_session - start_datetime_of_this_session) / (index_of_last_row + 1)) + first_datetime_of_this_MSN_session)
                            # このMSNの一番最初のセッションのStart Date + (1セッションあたりの時間 * 行番号) = このMSNの一番最初のセッションからの経過時間 
                            df_to_add['trial_time_delta'] = pd.Series((df_X[1] - 1) * ((end_datetime_of_this_session - start_datetime_of_this_session) / (index_of_last_row + 1)) + (start_datetime_of_this_session - first_datetime_of_this_MSN_session)).dt.total_seconds()
                            # subjectの名前
                            df_to_add['subject'] = a_subject_key
                            # msnの名前
                            df_to_add['msn'] = MSN_of_this_session
                            # 余分にnosepokeした時間 = 実際にnosepokeした時間 - 求められた時間
                            df_to_add['extra_nosepoke_time(s)'] = pd.to_timedelta(df_X[5] - df_X[4], unit='S').dt.total_seconds()
                            # nosepokeが成功したかどうか
                            df_to_add['success_nosepoke'] = (df_X[5] >= df_X[4])
                            # lever choiceが正しいかどうか
                            df_to_add.loc[df_Y[3] <= 2, 'correct_leverchoice'] = (df_Y[3] <= 1) | ~(df_Y[3] == 2)
                            # どのleverを選択したか
                            df_to_add['leverchoice:_right=1_left=-1_other=0'] = (df_Y[3] == 1)*1 + (df_Y[3] == 0)*(-1) + (df_Y[3] > 1)*0

                            # 以上の処理を行ったdataframeを結合し、行名列名を付ける
                            df_of_this_session = pd.concat([df_X.iloc[:, 1:], df_Y.iloc[:, 1:], df_to_add], axis='columns')
                            list_of_df_X_columns = ['order_of_trial', 'nosepoke_in_20min', 'nosepoke_latency', 'nosepoke_duration_required', 'nosepoke_duration_measured', 'empty']
                            list_of_df_X_columns = [f'X{i+1}_{list_of_df_X_columns[i]}' for i in range(len(list_of_df_X_columns))]
                            list_of_df_Y_columns = ['sd_presented', 'lever_pressed', 'outcome', 'sd_presentation_duration', 'reaction_time', 'lever_choice_pattern']
                            list_of_df_Y_columns = [f'Y{i+1}_{list_of_df_Y_columns[i]}' for i in range(len(list_of_df_Y_columns))]
                            list_of_df_to_add_columns = df_to_add.columns.to_list()
                            MultiColmins_of_df_of_this_session = pd.MultiIndex.from_product([[f'{a_subject_key}'], list_of_df_X_columns + list_of_df_Y_columns + list_of_df_to_add_columns], names=['subject', 'data'])
                            
                            MultiIndex_of_df_of_this_session = pd.MultiIndex.from_product([[MSN_of_this_session], df_of_this_session.index.to_list()], names=['MSN', 'index'])
                            ############################################################
                            #print('check point 1')
                            #if a_subject_key == 'I322':
                            #    display(df_of_this_session)
                            ############################################################
                            df_of_this_session = pd.DataFrame(df_of_this_session.values, index=MultiIndex_of_df_of_this_session, columns=MultiColmins_of_df_of_this_session)
# 処理終わり              
############################################################
                            # このdfをdf_of_this_subjectに結合する
                            df_of_this_subject = pd.concat([df_of_this_subject, df_of_this_session], axis='index')
                        # これで1つのsessionの処理が終わった

                    # このsubjectに関するまとめの処理の開始
                    if df_of_this_subject.empty:
                        print(f'{a_subject_key}: This rat didn`t do the target MSN')
############################################################
# DOEの弁別訓練のための処理        
                    else:
                        #print('check point 0')
                        #列名をMultiIndexにしてるので、行冥の指定には、DataFrame[level1の列名][level2の列名]という形で指定する
                        df_of_this_subject_nosepoke_isnot_false = df_of_this_subject[df_of_this_subject[a_subject_key]['correct_leverchoice'].notna()]
                        df_of_this_subject_nosepoke_isnot_false = df_of_this_subject_nosepoke_isnot_false.reset_index(drop=True)
                        df_of_this_subject = df_of_this_subject.reset_index(drop=True)
                        list_of_moving_average_columns = ['success_nosepoke_moving_average', 'correct_leverchoice_moving_average', 'leverchoice_bias_moving_average']
                        MultiColumns_of_df_of_this_subject = pd.MultiIndex.from_product([[f'{a_type_of_experiment.lower()}_{a_subject_key}'], list_of_moving_average_columns], names=['subject', 'data'])

                        # まずは、nosepokeが失敗したときを含む処理
                        df_of_analyse_result_of_this_subject = pd.DataFrame()
                        df_of_analyse_result_of_this_subject = pd.concat([df_of_analyse_result_of_this_subject, df_of_this_subject[a_subject_key]['success_nosepoke'].rolling(window_size, min_periods=1).mean()], axis='columns')                       
                        df_of_analyse_result_of_this_subject = pd.concat([df_of_analyse_result_of_this_subject, df_of_this_subject[a_subject_key]['correct_leverchoice'].rolling(window_size, min_periods=1).mean()], axis='columns')
                        df_of_analyse_result_of_this_subject = pd.concat([df_of_analyse_result_of_this_subject, df_of_this_subject[a_subject_key]['leverchoice:_right=1_left=-1_other=0'].rolling(window_size, min_periods=1).sum().abs() / df_of_this_subject[a_subject_key]['leverchoice:_right=1_left=-1_other=0'].abs().rolling(window_size, min_periods=1).sum()], axis='columns')
                        df_of_analyse_result_of_this_subject.columns = MultiColumns_of_df_of_this_subject
                        df_of_this_subject = pd.concat([df_of_this_subject, df_of_analyse_result_of_this_subject], axis='columns')
                        df_of_this_subject.to_excel(f'{path_to_xlsx_folder}/{a_subject_key}.({a_type_of_experiment}_window{window_size}_withNULL).xlsx')
                        #集計結果(nosepokeの失敗は不問)を、全体結果としてdfに追加
                        
                        ############################################################
                        #if a_subject_key == 'SH013' or a_subject_key == 'SH012':
                        #    print('check point 1')
                        #    display(df_of_this_subject.columns)
                        ############################################################
                        df_of_this_type_of_experiment = pd.concat([df_of_this_type_of_experiment, df_of_analyse_result_of_this_subject], axis='columns')
                        ############################################################
                        #if a_subject_key == 'SH013' or a_subject_key == 'SH012':
                        #    print('check point 2')
                        #    #print(df_of_this_subject.columns)
                        #    display(df_of_this_type_of_experiment.columns)
                        ############################################################
                        
                        
                        # 次に、nosepokeが失敗したときを除外した処理
                        df_of_analyse_result_of_this_subject_nosepoke_isnot_false = pd.DataFrame()
                        df_of_analyse_result_of_this_subject_nosepoke_isnot_false = pd.concat([df_of_analyse_result_of_this_subject_nosepoke_isnot_false, df_of_this_subject_nosepoke_isnot_false[a_subject_key]['success_nosepoke'].rolling(window_size, min_periods=1).mean()], axis='columns')
                        df_of_analyse_result_of_this_subject_nosepoke_isnot_false = pd.concat([df_of_analyse_result_of_this_subject_nosepoke_isnot_false, df_of_this_subject_nosepoke_isnot_false[a_subject_key]['correct_leverchoice'].rolling(window_size, min_periods=1).mean()], axis='columns')
                        df_of_analyse_result_of_this_subject_nosepoke_isnot_false = pd.concat([df_of_analyse_result_of_this_subject_nosepoke_isnot_false, df_of_this_subject_nosepoke_isnot_false[a_subject_key]['leverchoice:_right=1_left=-1_other=0'].rolling(window_size, min_periods=1).sum().abs() / df_of_this_subject_nosepoke_isnot_false[a_subject_key]['leverchoice:_right=1_left=-1_other=0'].abs().rolling(window_size, min_periods=1).sum()], axis='columns')
                        df_of_analyse_result_of_this_subject_nosepoke_isnot_false.columns = MultiColumns_of_df_of_this_subject
                        df_of_this_subject_nosepoke_isnot_false = pd.concat([df_of_this_subject_nosepoke_isnot_false, df_of_analyse_result_of_this_subject_nosepoke_isnot_false], axis='columns')
                        df_of_this_subject_nosepoke_isnot_false.to_excel(f'{path_to_xlsx_folder_nosepoke_isnot_false}/{a_subject_key}.({a_type_of_experiment}_window{window_size}_withoutNULL).xlsx')
                        #集計結果(nosepokeの失敗は除外)を、全体結果としてdfに追加
                        df_of_this_type_of_experiment_nosepoke_isnot_false = pd.concat([df_of_this_type_of_experiment_nosepoke_isnot_false, df_of_analyse_result_of_this_subject_nosepoke_isnot_false], axis='columns')
                        
                        print(f'{a_subject_key}, WindowSize{window_size}: \tDONE!! xlsx file has been exported!!')
                    # これでひとつのsubjectの処理が終わった
                # これでひとつのjsonの処理が終わった

        #list of df to make statusをループの前に置いてしまったので、最後のpd.concatが反映されてなかった。
        list_of_df_to_make_status = [df_of_this_type_of_experiment, df_of_this_type_of_experiment_nosepoke_isnot_false]
        for i in range(len(list_of_df_to_make_status)):
            a_df = list_of_df_to_make_status[i]
            # swaplevelは、順序を変更したデータのコピーを返すので、元のデータは変更されないらしい。
            a_df = a_df.swaplevel('subject', 'data', axis='columns')
            df_to_add = pd.DataFrame()
            for a_colmun in list_of_moving_average_columns:
                df_of_status = pd.DataFrame()
                df_of_status = pd.concat([df_of_status, a_df[a_colmun].mean(axis='columns')], axis='columns')
                df_of_status = pd.concat([df_of_status, a_df[a_colmun].std(axis='columns')], axis='columns')
                df_of_status = pd.concat([df_of_status, a_df[a_colmun].sem(axis='columns')], axis='columns') 
                MultiColmins_of_new_df = pd.MultiIndex.from_product([[a_colmun], [f'{a_type_of_experiment.lower()}_average', f'{a_type_of_experiment.lower()}_std', f'{a_type_of_experiment.lower()}_sem']], names=['data', 'subject'])
                df_of_status.columns = MultiColmins_of_new_df
                df_to_add = pd.concat([df_to_add, df_of_status], axis='columns')
            df_to_add = df_to_add.swaplevel('subject', 'data', axis='columns')
            list_of_df_to_make_status[i] = pd.concat([list_of_df_to_make_status[i], df_to_add], axis='columns')
            list_of_df_to_make_status[i] = list_of_df_to_make_status[i].swaplevel('data','subject',  axis='columns')
            list_of_df_to_make_status[i] = list_of_df_to_make_status[i].sort_index(axis='columns')

            
        #########################
        # めちゃくちゃ混乱した。
        # df_of_this_type_of_experimentと、list_of_df_to_make_status[0]は、同じオブジェクトだと思っていたが、違った。
        # 違うオブジェクトらしい。 
        # これが原因でdf_of_statusが反映されてなかった
        # リストに格納した時点で、違うオブジェクトになっているので、元のオブジェクトには反映されない。

        #print('check point 3')
        #display(list_of_df_to_make_status[0].columns)
        #display(df_of_this_type_of_experiment)
        #########################
# 処理終わり    
############################################################
        
        #print('check point 1')
        #display(list_of_df_to_make_status[0])
        df_of_this_folder = pd.concat([df_of_this_folder, list_of_df_to_make_status[0]], axis='columns')
        df_of_this_folder_nosepoke_isnot_false = pd.concat([df_of_this_folder_nosepoke_isnot_false, list_of_df_to_make_status[1]], axis='columns')
        # これでひとつのtype_of_experimentの処理が終わった

    # swaplevelは、順序を変更したデータのコピーを返すので、元のデータは変更されないらしい。
    df_of_this_folder = df_of_this_folder.sort_index(axis='columns', level='data')
    df_of_this_folder.to_excel(f'{path_to_xlsx_folder}/all_rats.(mod1_window{window_size}_withNULL).xlsx')
    df_of_this_folder.iloc[df_of_this_folder.index % window_size == (window_size - 1), :].to_excel(f'{path_to_xlsx_folder}/all_rats.(mod{window_size}_window{window_size}_withNULL).xlsx')
    
    df_of_this_folder_nosepoke_isnot_false = df_of_this_folder_nosepoke_isnot_false.sort_index(axis='columns', level='data')
    df_of_this_folder_nosepoke_isnot_false.to_excel(f'{path_to_xlsx_folder_nosepoke_isnot_false}/all_rats.(mod1_window{window_size}_withoutNULL).xlsx')
    df_of_this_folder_nosepoke_isnot_false.iloc[df_of_this_folder_nosepoke_isnot_false.index % window_size == (window_size - 1), :].to_excel(f'{path_to_xlsx_folder_nosepoke_isnot_false}/all_rats.(mod{window_size}_window{window_size}_withoutNULL).xlsx')
    # これでひとつのfolderの処理が終わった         

# 出力されたxlsxファイルの中身が、おかしくないか調べる関数
def check_the_number_of_trial_of_is_match_between_xlsx_and_memo(path_to_output_folder):
    path_to_json_data_folder = f'{path_to_output_folder}/json_from_RawData'
    path_of_all_data = sorted(glob.glob(path_to_json_data_folder + '/*[.json]')) 
    list_of_target = []
    for a_type in dict_of_target.keys():
        list_of_target.append([i.upper() for i in dict_of_target[a_type]])
    df_of_this_folder = pd.DataFrame()
    # ラットごとの、jsonファイルのパス
    for path_of_a_data in path_of_all_data:
        # ラット一匹分のjsonファイルを開く
        with open(path_of_a_data) as json_open:
            json_load = json.load(json_open)

            # 一番上の階層のkeyから、
            # ラットの名前を取得
            for key_of_a_subject in json_load.keys():
                if key_of_a_subject in list_of_target:
                    pass
                else:
                    break
                df_of_this_subject = pd.DataFrame()
                multiindex_of_this_folder = pd.MultiIndex.from_product([[key_of_a_subject], ['Start_Date', 'MSN', 'key_of_a_session', 'the_number_of_trial', 'sum_of_MSN']], names=['Subject', 'DATA'])
                first_MSN_sesssion_datetime = 0
                last_MSN_session = False
                sum_of_trial = 0

                # ２番目の階層のkeyから
                # セッションの名前（MSN_date）を取得
                for key_of_a_session in json_load[key_of_a_subject].keys():
                    json_of_this_session = json_load[key_of_a_subject][key_of_a_session]
                    MSN_of_this_session = json_of_this_session["MSN"]
                    start_datetime = dt.strptime(json_of_this_session["Start Date"] + json_of_this_session["Start Time"], '%m/%d/%y%H:%M:%S')
                
                    the_number_of_trail = 0
                    keys_of_data = json_of_this_session.keys()
                    for a_key in keys_of_data:

                        if isinstance(json_of_this_session[a_key], list):
                            if len(json_of_this_session[a_key]) == 1:
                                pass
                            elif isinstance(json_of_this_session[a_key][1], list):
                                a_df_of_this_session = pd.DataFrame(json_of_this_session[a_key]).astype(float)
                                index_of_last_row = 0
                                for index, row in a_df_of_this_session.iterrows():
                                    if row[1:].sum() != 0:
                                        index_of_last_row = index
                                    elif row[1:].sum() == 0:
                                        pass
                                edited_df_of_this_session = a_df_of_this_session.iloc[:index_of_last_row+1, :]
                                if the_number_of_trail <= len(edited_df_of_this_session):
                                    the_number_of_trail = len(edited_df_of_this_session)
                            
                    if MSN_of_this_session == last_MSN_session:
                        sum_of_trial += the_number_of_trail
                    elif MSN_of_this_session != last_MSN_session:
                        last_MSN_session = MSN_of_this_session
                        sum_of_trial = the_number_of_trail

                    start_datetime = str(start_datetime).replace(' ', '_').replace(':', '-')
                    a_df_of_this_session = pd.DataFrame([[start_datetime, MSN_of_this_session, key_of_a_session, the_number_of_trail, sum_of_trial]], columns=multiindex_of_this_folder)
                    df_of_this_subject = pd.concat([df_of_this_subject, a_df_of_this_session], axis=0)
                    df_of_this_subject.reset_index(drop=True, inplace=True)
                df_of_this_folder = pd.concat([df_of_this_folder, df_of_this_subject], axis=1)
                print(f'DONE!: {key_of_a_subject}')
    #display(df_of_this_folder)
    df_of_this_folder.to_excel(f'{path_to_output_folder}/the_number_of_trial.xlsx')


def make_graph_from_xlsx(path_to_output_folder):
    path_to_graph_folder = path_to_output_folder + '/graph'
    if not os.path.exists(path_to_graph_folder):
        os.mkdir(path_to_graph_folder)
    elif os.path.exists(path_to_graph_folder):
        shutil.rmtree(path_to_graph_folder)
        os.mkdir(path_to_graph_folder)
    
    list_of_path_to_xlsx_folder = sorted(glob.glob(path_to_output_folder + '/xlsx_from_json' + '*'))

    #print('check point 1')
    #print(list_of_path_to_xlsx_folder)

    for a_path_to_xlsx in list_of_path_to_xlsx_folder:
        # print(a_path)
        list_of_path_to_all_rats_xlsx = sorted(glob.glob(a_path_to_xlsx + '/all_rats' + '*' + '.xlsx'))
        variation_of_this_folder = a_path_to_xlsx.split('.')[1]
        largest_mod_num = 0
        a_path_to_all_rats_xlsx_largest_mod = ''

        #modが最大のxlsxファイルを選択
        for a_path_to_all_rats in list_of_path_to_all_rats_xlsx:
            mod_num = re.findall('mod(\d+)', a_path_to_all_rats)
            if int(mod_num[0]) > largest_mod_num:
                largest_mod_num = int(mod_num[0])
                a_path_to_all_rats_xlsx_largest_mod = a_path_to_all_rats

        # header=[0,1]で、1行目と2行目をヘッダーとして読み込む
        df_of_all_rats = pd.read_excel(a_path_to_all_rats_xlsx_largest_mod, header=[0,1], index_col=[0])

        # make figure
        list_of_data = df_of_all_rats.columns.get_level_values('data').unique().to_list()
        list_of_subject = df_of_all_rats.columns.get_level_values('subject').unique().to_list()
        list_of_type_of_experiment = sorted(set([i.split('_')[0].upper() for i in df_of_all_rats.columns.get_level_values('subject').unique().tolist()]))
        color_map = [plt.cm.tab20(i) for i in range(len(list_of_type_of_experiment) * 2)]
        color_map_index = 0
        fig, axis = plt.subplots(1, len(list_of_data), figsize=(30, 10), dpi=300)
        for a_data_No in range(len(list_of_data)):
            a_data = list_of_data[a_data_No]
            for a_type_No in range(len(list_of_type_of_experiment)):
                a_type = list_of_type_of_experiment[a_type_No].upper()
                
                list_of_this_type_subject = [i for i in list_of_subject if i.split('_')[0].upper() == a_type]
                list_of_this_type_subject.sort()
                for a_subject in list_of_this_type_subject:
                    df_of_this_type_subject = df_of_all_rats[a_data][a_subject]
                    if re.findall('average', a_subject):
                        axis[a_data_No].plot(df_of_this_type_subject, color=color_map[color_map_index], label=a_subject, alpha=1.0)
                    elif not re.findall('std', a_subject) and not re.findall('sem', a_subject):
                        axis[a_data_No].plot(df_of_this_type_subject, color=color_map[color_map_index + 1], label=a_subject, alpha=0.2)
                color_map_index += 2
            color_map_index = 0

        for a_data_No in range(len(list_of_data)):
            a_data = list_of_data[a_data_No]
            colmun = a_data.split('_moving_average')[0]
            axis[a_data_No].set_title(f'{colmun}_{variation_of_this_folder}')
            axis[a_data_No].legend()
            axis[a_data_No].set_xlabel('traial')
            axis[a_data_No].set_ylabel('percentage (%)')
        # テキストをテキストとして出力する設定
        plt.rc("svg", fonttype="none")
        # SVGとして保存
        fig.savefig(f'{path_to_graph_folder}/{variation_of_this_folder}.svg', format='svg')
        # PNGとして保存
        fig.savefig(f'{path_to_graph_folder}/{variation_of_this_folder}.png', format='png')

In [ ]:
for i in [path_to_RawData_folder]:
    make_json_from_RawData(i)
    repaire_known_issue_of_json(i)
    for j in window_size:
        make_xlsx_about_DOE_from_json(i, dict_of_target, j)

In [ ]:
make_graph_from_xlsx(path_to_RawData_folder)